# Разработка дашборда событий для агрегатора новостей

Аналитика в Яндекс.Дзене строится на базе пользовательских взаимодействий с карточками статей.
Каждую карточку определяют её тема и источник. Пользователей системы также характеризует возрастная категория. 

Необходимо разработать дашборд, отвечающий на следующие вопросы:
- Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
- Как много карточек генерируют источники с разными темами?
- Как соотносятся темы карточек и темы источников?

Дашборд будет основываться на пайплайне, который будет брать данные из таблицы, в которых хранятся сырые данные, трансформировать данные и сохранять в итоговую таблицу.

## Получение данных

Для начала подключимся к базе данных, импортировав нужные библиотеки.

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# создаем коннекцию к базе данных
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

Затем подготовим sql-запрос.

In [3]:
# задаем sql-запрос
query = 'SELECT * FROM dash_visits'

import 

In [4]:
# выполняем sql-запрос и записываем результат в переменную data
data = pd.io.sql.read_sql(query, con = engine) 

Посмотрим на результат.

In [6]:
# выводим основную информацию
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


Полученная таблица содержит все необходимое для дальнейшей работы.

## Обработка данных

Сгруппируем полученные данные по дате, темам и возрастному сегменту.

In [7]:
# считаем сумму "визитов" в группировке по дате, темам карточки/источника и возрастному сегменту
data_grouped = data.groupby(['item_topic', 'dt', 'source_topic', 'age_segment']).agg({'visits': 'sum'})

data_grouped 

visits
item_topic dt                  source_topic  age_segment        
Деньги     2019-09-24 18:28:00 Деньги        36-40             1
                               Дети          18-25             1
                                             31-35             1
                                             36-40             1
                                             41-45             1
...                                                          ...
Юмор       2019-09-24 19:00:00 Строительство 26-30             1
                                             45+               1
                               Технологии    31-35             1
                               Финансы       36-40             1
                                             41-45             1

[30745 rows x 1 columns]

Сохраним полученные данные в csv-файл.

In [8]:
# записываем данные в файл "dash_visits.csv"
data_grouped.to_csv('dash_visits.csv')

In [9]:
# для проверки считаем сумму числа событий
data_grouped['visits'].sum()

310207

Дальнейшая обработка данных из файла осуществляется в Tableau Publiс. 

Полученный дашборд доступен по ссылке: https://public.tableau.com/app/profile/bazedius/viz/Eventsdashboard/sheet4?publish=yes 